# 64-Band Real-Time Graphical Equalizer
This notebook records audio from the microphone, applies a 64-band graphical equalizer in real-time, and plays it back using sounddevice.

In [1]:
!pip install sounddevice numpy scipy ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 20.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]


In [2]:
import numpy as np
import sounddevice as sd
from scipy.signal import butter, sosfilt
import ipywidgets as widgets
from IPython.display import display
import threading

# Audio parameters
fs = 44100
blocksize = 1024
channels = 1

# Frequency bands (log spaced)
freqs = np.logspace(np.log10(20), np.log10(20000), 65)

# Create sliders
sliders = []
for i in range(64):
    slider = widgets.FloatSlider(
        value=0.0,
        min=-12.0,
        max=12.0,
        step=0.5,
        description=f'{int(freqs[i])}Hz',
        orientation='vertical',
        layout=widgets.Layout(height='200px', width='30px')
    )
    sliders.append(slider)

display(widgets.HBox(sliders))

# Design band filters
sos_filters = []
for i in range(64):
    low = freqs[i] / (fs/2)
    high = freqs[i+1] / (fs/2)
    sos = butter(2, [low, high], btype='band', output='sos')
    sos_filters.append(sos)

# Filter states
states = [np.zeros((sos.shape[0], 2)) for sos in sos_filters]

def audio_callback(indata, outdata, frames, time, status):
    global states
    x = indata[:, 0]
    y = np.zeros_like(x)
    
    for i in range(64):
        gain = 10**(sliders[i].value / 20)
        filtered = sosfilt(sos_filters[i], x, zi=states[i])[0]
        y += gain * filtered
    
    outdata[:, 0] = y

stream = sd.Stream(
    samplerate=fs,
    blocksize=blocksize,
    channels=channels,
    callback=audio_callback
)

def start_audio():
    stream.start()

def stop_audio():
    stream.stop()

start_button = widgets.Button(description='Start')
stop_button = widgets.Button(description='Stop')

start_button.on_click(lambda x: start_audio())
stop_button.on_click(lambda x: stop_audio())

display(widgets.HBox([start_button, stop_button]))

In [3]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048
NUM_BANDS = 64

# Global gains array (1.0 = 0dB)
# We use a numpy array so the audio callback can read it efficiently
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    # Perform FFT
    spectrum = rfft(indata[:, 0])
    
    # Map the 64 gain bands to the FFT bins
    # We use linear interpolation for speed
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    # Apply gains and invert back to time domain
    processed_spectrum = spectrum * applied_gains
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic ---
sliders = []
def update_gain(change):
    # 'owner' is the slider that triggered the change
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

# Create 64 vertical sliders
for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0,
        min=0.0,
        max=3.0,  # Max 3x boost
        step=0.1,
        orientation='vertical',
        readout=False,
        layout=widgets.Layout(width='15px', height='200px')
    )
    # Store the index in a custom attribute to identify it in the callback
    slider.description_int = i 
    slider.observe(update_gain, names='value')
    sliders.append(slider)

ui = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll'))

# --- 3. Run the Stream ---
print("Adjust the sliders to change the EQ in real-time.")
display(ui)

try:
    with sd.Stream(samplerate=SAMPLE_RATE,
                   blocksize=BLOCK_SIZE,
                   channels=1,
                   callback=audio_callback):
        print("Audio Stream Active. Press 'Stop' in Jupyter to end.")
        # Keeps the cell running while the background thread handles audio
        while True:
            sd.sleep(1000)
except KeyboardInterrupt:
    print("\nStream stopped.")

Adjust the sliders to change the EQ in real-time.


Audio Stream Active. Press 'Stop' in Jupyter to end.

Stream stopped.


In [4]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048
NUM_BANDS = 64
global_stream = None  # To hold the stream object

# Global gains array (1.0 = 0dB)
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    spectrum = rfft(indata[:, 0])
    
    # Map gains to frequency bins
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    processed_spectrum = spectrum * applied_gains
    
    # Inverse FFT to time domain
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic ---
sliders = []
def update_gain(change):
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='12px', height='150px')
    )
    slider.description_int = i
    slider.observe(update_gain, names='value')
    sliders.append(slider)

# Group sliders
eq_box = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll', width='100%'))

# Create a Stop Button
stop_btn = widgets.Button(
    description='Stop Audio',
    button_style='danger', # Red color
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Audio Stream Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 3. Start the Stream ---
# We use .start() instead of a 'with' block so the cell can finish execution
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)

_IncompleteInputError: incomplete input (3145588784.py, line 78)

In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048
NUM_BANDS = 64
global_stream = None  # To hold the stream object

# Global gains array (1.0 = 0dB)
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    spectrum = rfft(indata[:, 0])
    
    # Map gains to frequency bins
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    processed_spectrum = spectrum * applied_gains
    
    # Inverse FFT to time domain
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic ---
sliders = []
def update_gain(change):
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='12px', height='150px')
    )
    slider.description_int = i
    slider.observe(update_gain, names='value')
    sliders.append(slider)

# Group sliders
eq_box = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll', width='100%'))

# Create a Stop Button
stop_btn = widgets.Button(
    description='Stop Audio',
    button_style='danger', # Red color
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Audio Stream Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 3. Start the Stream ---
# We use .start() instead of a 'with' block so the cell can finish execution
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)
    global_stream.start()
    
    print("Audio is live! Adjust sliders below.")
    display(stop_btn)
    display(eq_box)
    
except Exception as e:
    print(f"Error starting stream: {e}")

Audio is live! Adjust sliders below.


Button(button_style='danger', description='Stop Audio', icon='square', style=ButtonStyle())

In [1]:
import ipywidgets as w; w.IntSlider()

IntSlider(value=0)

In [2]:
w.IntSlider()

IntSlider(value=0)

In [3]:
a = w.IntSlider()

In [4]:
a

IntSlider(value=0)

In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
# Wavelets need larger blocks to avoid edge artifacts and overhead
# 4096 is safer for real-time Python performance with WPD
BLOCK_SIZE = 4096  
NUM_BANDS = 64
WAVELET_NAME = 'db5'  # 'db1' (Haar) is fastest. 'db4' is smoother but slower.

global_stream = None 
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Wavelet Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    # 1. Flatten input to 1D array for PyWavelets
    signal = indata[:, 0]
    
    # 2. Create Wavelet Packet Object
    # 'symmetric' padding reduces edge artifacts slightly
    wp = pywt.WaveletPacket(data=signal, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
    
    # 3. Get the 64 leaf nodes ordered by FREQUENCY
    # (Natural wavelet order is not frequency ordered)
    nodes = wp.get_level(6, order='freq')
    
    # 4. Apply Gains
    # We iterate through our 64 sliders and multiply the node data
    # We use min() to ensure we don't crash if node count differs slightly (rare)
    for i in range(min(len(nodes), len(gains))):
        nodes[i].data = nodes[i].data * gains[i]
        
    # 5. Reconstruct
    # update=False prevents re-computing coefficients we just modified
    new_signal = wp.reconstruct(update=False)
    
    # 6. Safety Check & Reshape
    # Reconstruction might produce slightly different length due to padding
    if len(new_signal) != len(signal):
        # Truncate or pad if necessary (usually just a 1-sample diff)
        new_signal = new_signal[:len(signal)]
        
    outdata[:, 0] = new_signal

# --- 2. The UI Logic (Same as before) ---
sliders = []
def update_gain(change):
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='12px', height='150px')
    )
    slider.description_int = i
    slider.observe(update_gain, names='value')
    sliders.append(slider)

eq_box = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll', width='100%'))

stop_btn = widgets.Button(
    description='Stop WPD Audio',
    button_style='danger',
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Wavelet Stream Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 3. Start the Stream ---
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    print(f"Starting Wavelet Stream ({WAVELET_NAME}, {NUM_BANDS} bands)...")
    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)
    global_stream.start()
    
    print("Audio is live! Processing with Wavelet Packets.")
    display(stop_btn)
    display(eq_box)
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Wavelet Stream (db5, 64 bands)...
Audio is live! Processing with Wavelet Packets.


Button(button_style='danger', description='Stop WPD Audio', icon='square', style=ButtonStyle())

In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048 
NUM_BANDS = 64
global_stream = None 

# Global gains array
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. Audio Callback (FFT Method) ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    spectrum = rfft(indata[:, 0])
    
    # Map gains to frequency bins (Linear interpolation)
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    processed_spectrum = spectrum * applied_gains
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic with Labels ---
slider_containers = []

def update_gain(change):
    # 'owner' is the slider object
    new_val = change['new']
    # We stored the index and the label widget inside the slider object for easy access
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    
    # Update global gain array
    gains[band_index] = new_val
    
    # Update the label text
    label_widget.value = f"{new_val:.1f}"

for i in range(NUM_BANDS):
    # Create the Label
    lbl = widgets.Label(
        value="1.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    
    # Create the Slider
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    
    # Link them together
    slider.band_index = i
    slider.label_ref = lbl  # Store reference to label inside slider
    slider.observe(update_gain, names='value')
    
    # Stack them vertically (Label on top, Slider below)
    # We use VBox to group them
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

# Group all 64 columns into one scrollable row
eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Audio',
    button_style='danger',
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    stop_btn.disabled = True
    print("Stream Stopped.")

stop_btn.on_click(stop_stream)

# --- 4. Start Stream ---
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)
    global_stream.start()
    
    print("Audio Live. Gains displayed above sliders.")
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error: {e}")

Audio Live. Gains displayed above sliders.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096  # Larger block size for stable Wavelet processing
NUM_BANDS = 64
WAVELET_NAME = 'db1' # 'db1' is fast. Try 'db4' for smoother filtering if CPU allows.
NOISE_VOLUME = 0.2   # Safety volume (White noise is loud!)

global_stream = None 

# Global gains array (Start at 1.0)
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The White Noise + Wavelet Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    # A. Generate White Noise (Uniform distribution between -1 and 1)
    # We ignore 'indata' entirely.
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    # B. Wavelet Decomposition
    try:
        # Create Wavelet Packet tree
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # C. Apply Gains
        # Ensure we don't exceed bounds if node count varies slightly
        count = min(len(nodes), len(gains))
        for i in range(count):
            # Multiply the coefficients of this band by the slider value
            nodes[i].data = nodes[i].data * gains[i]
            
        # D. Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # E. Safety Clipping & Buffer Filling
        # Wavelet reconstruction length can vary by a few samples due to padding
        # We truncate or pad to match the requested 'frames' size
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        # Write to output (Mono)
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic (Sliders + Labels) ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    
    # Update global array
    gains[band_index] = new_val
    # Update label text
    label_widget.value = f"{new_val:.1f}"

# Create 64 Sliders with Labels
for i in range(NUM_BANDS):
    # Label
    lbl = widgets.Label(
        value="1.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    
    # Linkage
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    # Stack vertically
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

# Scrollable Container
eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting White Noise Generator with {NUM_BANDS}-band Wavelet EQ...")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    # We use an OutputStream since we are generating data, not capturing it.
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting White Noise Generator with 64-band Wavelet EQ...


Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x7f3efc61aa30>:
Traceback (most recent call last):
  File "/home/vruiz/envs/intercom/lib/python3.14/site-packages/sounddevice.py", line 878, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
  File "/home/vruiz/envs/intercom/lib/python3.14/site-packages/sounddevice.py", line 2777, in _wrap_callback
    callback(*args)
TypeError: audio_callback() missing 1 required positional argument: 'status'


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096  # Larger block size for stable Wavelet processing
NUM_BANDS = 64
WAVELET_NAME = 'db1' # 'db1' is fast.
NOISE_VOLUME = 0.1   # Safety volume

global_stream = None 
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Corrected Callback (No 'indata') ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Generate White Noise (Uniform distribution)
    # We generate exactly 'frames' worth of noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        # Wavelet reconstruction length can vary by a few samples due to padding
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        # Write to output (Mono)
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic (Same as before) ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.1f}"

for i in range(NUM_BANDS):
    lbl = widgets.Label(
        value="1.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting White Noise Generator with {NUM_BANDS}-band Wavelet EQ...")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    # Using OutputStream (Out Only)
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting White Noise Generator with 64-band Wavelet EQ...


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# --- CHANGE 1: Initialize gains to 0.0 ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.1f}"

for i in range(NUM_BANDS):
    # --- CHANGE 2: Label starts at "0.0" ---
    lbl = widgets.Label(
        value="0.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    
    # --- CHANGE 3: Slider starts at 0.0 ---
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting Silent Stream ({NUM_BANDS}-band Wavelet EQ). Raise sliders to hear noise.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")


Starting Silent Stream (64-band Wavelet EQ). Raise sliders to hear noise.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# Initialize gains to 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        nodes = wp.get_level(6, order='freq')
        
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        processed_signal = wp.reconstruct(update=False)
        
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    # Show 2 decimal places for higher resolution
    label_widget.value = f"{new_val:.2f}"

for i in range(NUM_BANDS):
    lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # --- CHANGE: Taller sliders (400px) and finer step (0.01) ---
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting High-Res Silent Stream. Raise sliders to hear noise.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting High-Res Silent Stream. Raise sliders to hear noise.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# Initialize gains to 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        nodes = wp.get_level(6, order='freq')
        
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        processed_signal = wp.reconstruct(update=False)
        
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

# Calculate bandwidth per band
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.2f}"

for i in range(NUM_BANDS):
    # Top Label (Gain)
    gain_lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # Slider (Long & Fine)
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    
    # Bottom Label (Frequency)
    # Calculate start freq of this band
    freq_start = i * bandwidth
    
    # Format: 1000+ uses "k" (e.g., 1.2k), otherwise just Hz
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(
        value=freq_text,
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    # Linkage
    slider.band_index = i
    slider.label_ref = gain_lbl
    slider.observe(update_gain, names='value')
    
    # Stack: Gain -> Slider -> Freq
    col = widgets.VBox(
        [gain_lbl, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting Stream. Bandwidth per slider: ~{int(bandwidth)}Hz")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Stream. Bandwidth per slider: ~344Hz


In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# Initialize gains to 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# Global Mute Flag
is_muted = False

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Check if muted
    if is_muted:
        outdata.fill(0)
        return

    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

# Calculate bandwidth per band
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.2f}"

for i in range(NUM_BANDS):
    # Top Label (Gain)
    gain_lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    
    # Bottom Label (Frequency)
    freq_start = i * bandwidth
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(
        value=freq_text,
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    slider.band_index = i
    slider.label_ref = gain_lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([gain_lbl, slider, freq_lbl], 
                       layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Mute Button Logic ---
mute_btn = widgets.Button(
    description='Mute',
    button_style='danger', # Red initially
    icon='volume-off'
)

def toggle_mute(b):
    global is_muted
    is_muted = not is_muted
    
    if is_muted:
        mute_btn.description = "Unmute"
        mute_btn.button_style = 'success' # Green
        mute_btn.icon = 'volume-up'
    else:
        mute_btn.description = "Mute"
        mute_btn.button_style = 'danger' # Red
        mute_btn.icon = 'volume-off'

mute_btn.on_click(toggle_mute)

# --- 4. Start Execution ---
print(f"Starting Stream. Use the button below to Mute/Unmute.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([mute_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Stream. Use the button below to Mute/Unmute.


In [3]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# --- Global State ---
# Gains: Start at 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)
# Band Active Status: Start as True (Unmuted)
band_active = np.ones(NUM_BANDS, dtype=bool) 
# Global Mute
global_mute = False

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Global Mute Check
    if global_mute:
        outdata.fill(0)
        return

    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains with Mute Logic
        count = min(len(nodes), NUM_BANDS)
        for i in range(count):
            # If band is active, use slider gain. If muted, gain is 0.
            current_gain = gains[i] if band_active[i] else 0.0
            nodes[i].data = nodes[i].data * current_gain
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

# Handler for Slider changes
def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.2f}"

# Handler for Band Mute Buttons
def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new'] # True = Active, False = Muted
    
    # Update global state
    band_active[idx] = is_active
    
    # Update Button Appearance
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success' # Green
        btn.tooltip = "Click to Mute Band"
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'  # Red
        btn.tooltip = "Click to Unmute Band"

# Build the 64-band UI
for i in range(NUM_BANDS):
    
    # A. Per-Band Mute Button (ToggleButton)
    # Value=True means Active (Unmuted)
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success', # Starts Green
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')

    # B. Gain Label
    gain_lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # C. Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    slider.label_ref = gain_lbl
    slider.observe(update_gain, names='value')
    
    # D. Frequency Label
    freq_start = i * bandwidth
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(
        value=freq_text,
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    # Stack: Mute Btn -> Gain -> Slider -> Freq
    col = widgets.VBox(
        [mute_btn, gain_lbl, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---
master_mute_btn = widgets.Button(
    description='Global Mute',
    button_style='warning',
    icon='volume-off'
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='danger',
    icon='square'
)

def toggle_global_mute(b):
    global global_mute
    global_mute = not global_mute
    if global_mute:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'danger'
    else:
        master_mute_btn.description = "Global Mute"
        master_mute_btn.button_style = 'warning'

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True

master_mute_btn.on_click(toggle_global_mute)
stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting Wavelet EQ.")
print("Top Button: Mute specific band | Slider: Gain | Bottom: Frequency")

try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Display Controls
    controls = widgets.HBox([master_mute_btn, stop_btn])
    display(widgets.VBox([controls, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Wavelet EQ.
Top Button: Mute specific band | Slider: Gain | Bottom: Frequency


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
# We track the active state of each band here
band_active = np.ones(NUM_BANDS, dtype=bool) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Note: No global mute check here anymore. 
    # We rely entirely on the individual band_active states.

    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), NUM_BANDS)
        for i in range(count):
            # Check if THIS specific band is active
            if band_active[i]:
                current_gain = gains[i]
            else:
                current_gain = 0.0
                
            nodes[i].data = nodes[i].data * current_gain
            
        # Reconstruct
        processed_signal = wp.reconstruct(update=False)
        
        # Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []  # List to store the 64 button objects
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

# Update Gain Array
def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.2f}"

# Update Band Active State (Individual)
def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new'] # True=Active, False=Muted
    
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success' # Green
        btn.tooltip = "Band Active"
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'  # Red
        btn.tooltip = "Band Muted"

# Build UI
for i in range(NUM_BANDS):
    
    # Mute Button
    mute_btn = widgets.ToggleButton(
        value=True, # Start Active
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    
    # Store reference so Master button can find it later
    band_mute_widgets.append(mute_btn)

    # Gain Label
    gain_lbl = widgets.Label(value="0.00", layout=widgets.Layout(width='30px', justify_content='center'))
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    slider.label_ref = gain_lbl
    slider.observe(update_gain, names='value')
    
    # Freq Label
    freq_start = i * bandwidth
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(value=freq_text, layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px'))
    
    col = widgets.VBox([mute_btn, gain_lbl, slider, freq_lbl], 
                       layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Logic ---

# Master Button
master_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='120px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

# State variable for the master button
are_all_muted = False

def master_toggle_action(b):
    global are_all_muted
    
    # Toggle state
    are_all_muted = not are_all_muted
    
    if are_all_muted:
        # We want to MUTE everything
        target_value = False 
        master_btn.description = "Unmute All"
        master_btn.button_style = 'success'
        master_btn.icon = 'volume-up'
    else:
        # We want to UNMUTE everything
        target_value = True
        master_btn.description = "Mute All"
        master_btn.button_style = 'danger'
        master_btn.icon = 'volume-off'
        
    # Apply to all 64 buttons
    # Changing .value triggers the observe() function on each button automatically
    for btn in band_mute_widgets:
        btn.value = target_value

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_btn.disabled = True

master_btn.on_click(master_toggle_action)
stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting System.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls = widgets.HBox([master_btn, stop_btn])
    display(widgets.VBox([controls, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System.


In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), NUM_BANDS)
        for i in range(count):
            if band_active[i]:
                current_gain = gains[i]
            else:
                current_gain = 0.0
            nodes[i].data = nodes[i].data * current_gain
            
        # Reconstruct
        processed_signal = wp.reconstruct(update=False)
        
        # Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

# Update Gain Array
def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    # We only need to update the global array. 
    # The Text Box <-> Slider sync is handled by jslink.
    gains[band_index] = new_val

# Update Band Active State
def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

# Build UI
for i in range(NUM_BANDS):
    
    # A. Mute Button
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # B. Editable Float Box (Replaces Label)
    # We use BoundedFloatText to ensure users don't type -100 or 1000
    gain_box = widgets.BoundedFloatText(
        value=0.0,
        min=0.0,
        max=5.0,
        step=0.01,
        layout=widgets.Layout(width='45px', height='30px') # Slightly wider to fit numbers
    )
    
    # C. Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    
    # --- KEY CHANGE: Sync Slider and Text Box ---
    # jslink handles the UI update on the client side (very fast)
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    
    # We only need to observe one of them to update the audio engine
    slider.observe(update_gain, names='value')
    
    # D. Freq Label
    freq_start = i * bandwidth
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    # Stack
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Logic ---
master_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='120px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

are_all_muted = False

def master_toggle_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    
    if are_all_muted:
        target_value = False 
        master_btn.description = "Unmute All"
        master_btn.button_style = 'success'
        master_btn.icon = 'volume-up'
    else:
        target_value = True
        master_btn.description = "Mute All"
        master_btn.button_style = 'danger'
        master_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_btn.disabled = True

master_btn.on_click(master_toggle_action)
stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting Editable Wavelet EQ.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls = widgets.HBox([master_btn, stop_btn])
    display(widgets.VBox([controls, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Editable Wavelet EQ.


In [3]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2  # Volume for White Noise
TONAL_VOLUME = 0.05 # Volume for Tonal (Sine waves sum up loudly, so we keep this lower)

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False # False = Noise, True = Tonal

# --- Pre-calculate Center Frequencies for Tonal Mode ---
# Wavelet Packet leaves are linearly spaced
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
# Center of band i is: start + half_bandwidth
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])

# Phase accumulator for sine generation (to avoid clicks between blocks)
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    outdata.fill(0)
    
    # === MODE A: TONAL (Sum of Sine Waves) ===
    if is_tonal_mode:
        # Time array for this block
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        
        # We only generate sines for bands that have Gain > 0 AND are Active
        # This saves CPU and is how an EQ works
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                # Calculate frequency and gain
                freq = center_freqs[i]
                gain = gains[i] * TONAL_VOLUME
                
                # Generate sine wave for this band
                # sin(2*pi*f*t + phase)
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                
                # Add to total output
                output_signal += sine_wave
                
                # Update phase for next block
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal

    # === MODE B: NOISE (Wavelet Filtering) ===
    else:
        # Generate White Noise
        noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
        
        try:
            # Wavelet Decomposition
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            # Apply Gains
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0 # Mute
                
            # Reconstruct
            processed_signal = wp.reconstruct(update=False)
            
            # Buffer Filling
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            # print(f"Error: {e}") # Suppress print to avoid lag
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

# Update Gain Array
def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    # We only need to update the global array. 
    gains[band_index] = new_val

# Update Band Active State
def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

# Build UI
for i in range(NUM_BANDS):
    
    # Mute Button
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    
    # Sync Slider <-> Box
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    # Freq Label
    if center_freqs[i] >= 1000:
        freq_text = f"{center_freqs[i]/1000:.1f}k"
    else:
        freq_text = f"{int(center_freqs[i])}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    # Stack
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---
master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='120px')
)

# New Mode Switch Button
mode_btn = widgets.ToggleButton(
    value=False, # False = Noise
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='140px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    
    if are_all_muted:
        target_value = False 
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        target_value = True
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning' # Orange for Tonal
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'    # Blue for Noise

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True

master_mute_btn.on_click(master_mute_action)
stop_btn.on_click(stop_stream)
mode_btn.observe(toggle_mode, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Use 'Mode' button to switch between White Noise and Sine Waves.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls = widgets.HBox([master_mute_btn, mode_btn, stop_btn])
    display(widgets.VBox([controls, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Use 'Mode' button to switch between White Noise and Sine Waves.


output underflow
output underflow


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2
TONAL_VOLUME = 0.05

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

# 1. End Frequencies for Labels (Upper limit of each band)
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])

# 2. Center Frequencies for Tonal Generation (still needed for the sine wave pitch)
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])

phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    # Clear buffer completely to ensure no residual audio
    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    # If Tonal Mode is ON, we generate sines and RETURN immediately.
    # The code below this block will NOT execute.
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        
        for i in range(NUM_BANDS):
            # Only calculate if band is active and has volume
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * TONAL_VOLUME
                
                # Generate sine
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return  # <--- CRITICAL: Stop here. Do not touch noise code.

    # === MODE B: NOISE ONLY ===
    # This `else` block only runs if is_tonal_mode is False.
    else:
        noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    # --- FREQUENCY LABEL CHANGE ---
    # Now showing the END frequency of the band
    freq_val = end_freqs[i]
    
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---
master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='120px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='140px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    # Reset phases to avoid discontinuities when switching back
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True

master_mute_btn.on_click(master_mute_action)
stop_btn.on_click(stop_stream)
mode_btn.observe(toggle_mode, names='value')

# --- 4. Start Execution (Clean Start) ---
print(f"Starting System. Bottom labels now show the END frequency of each band.")
try:
    # Aggressive Cleanup
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop() # Stop any background sounddevice streams
    time.sleep(0.2) # Give OS a moment to release audio device

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls = widgets.HBox([master_mute_btn, mode_btn, stop_btn])
    display(widgets.VBox([controls, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Bottom labels now show the END frequency of each band.


In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base "Safe" Volumes (These prevent clipping when Master is at 100%)
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 # Master Volume (0.0 to 1.0)

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        
        # Calculate effective volume for this block
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                # Apply Master Volume here
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        # Generate Noise with Master Volume applied immediately
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    # Freq Label (End of Band)
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

# A. Button Row
master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='120px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='140px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

# B. Master Volume Slider (NEW)
master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', # Display as percentage (e.g. 80%)
    layout=widgets.Layout(width='400px')
)

are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True

master_mute_btn.on_click(master_mute_action)
stop_btn.on_click(stop_stream)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Use 'Master Vol' to control overall loudness.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Layout: Buttons on Top, Slider below, then EQ
    controls_row = widgets.HBox([master_mute_btn, mode_btn, stop_btn])
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row, vol_row, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Use 'Master Vol' to control overall loudness.
output underflow


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db5'

# Base "Safe" Volumes 
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    # --- FIX: Hard Mute Check ---
    # If Master Volume is effectively 0, mute everything immediately.
    if global_volume <= 0.001:
        outdata.fill(0)
        # We still need to advance phases for tonal mode to avoid
        # a "pop" when volume comes back up, but for now silence is key.
        return

    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        
        # Calculate effective volume
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        # Generate Noise
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    # Freq Label
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='120px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='140px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True

master_mute_btn.on_click(master_mute_action)
stop_btn.on_click(stop_stream)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Master Volume at 0% now guarantees silence.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls_row = widgets.HBox([master_mute_btn, mode_btn, stop_btn])
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row, vol_row, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Master Volume at 0% now guarantees silence.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base "Safe" Volumes 
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 
slider_widgets_list = [] # Store references to sliders to update them

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    # --- Hard Mute Check ---
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    # Store reference for Max/Min buttons
    slider_widgets_list.append(slider)
    
    # Freq Label
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

# A. Control Buttons
master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='120px')
)

# New: Max/Min Buttons
max_all_btn = widgets.Button(
    description='Max All',
    button_style='primary',
    icon='arrow-up',
    layout=widgets.Layout(width='100px')
)

min_all_btn = widgets.Button(
    description='Min All',
    button_style='primary',
    icon='arrow-down',
    layout=widgets.Layout(width='100px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

# B. Master Volume Slider
master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

# --- Logic for Buttons ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    # Set all gains to 5.0
    for s in slider_widgets_list:
        s.value = 5.0

def set_min_all(b):
    # Set all gains to 0.0
    for s in slider_widgets_list:
        s.value = 0.0

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    
    # Disable controls
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True

# Link Buttons
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Updated Layout
    controls_row = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, mode_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='10px')
    )
    
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row, vol_row, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System.


In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base "Safe" Volumes 
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 
slider_widgets_list = [] 

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    # --- Hard Mute Check ---
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    slider_widgets_list.append(slider)
    
    # Freq Label
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='120px')
)

max_all_btn = widgets.Button(
    description='Max All',
    button_style='primary',
    icon='arrow-up',
    layout=widgets.Layout(width='100px')
)

min_all_btn = widgets.Button(
    description='Min All',
    button_style='primary',
    icon='arrow-down',
    layout=widgets.Layout(width='100px')
)

print_btn = widgets.Button(
    description='Print Gains',
    button_style='success',
    icon='print',
    layout=widgets.Layout(width='120px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

# Output area for printing
out_log = widgets.Output()

# --- Logic ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    for s in slider_widgets_list:
        s.value = 5.0

def set_min_all(b):
    for s in slider_widgets_list:
        s.value = 0.0

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def print_gains_action(b):
    with out_log:
        clear_output() # Clear previous prints
        output_list = []
        for i in range(NUM_BANDS):
            # Format: (Frequency Hz, Gain)
            # We use the Center Frequency as requested
            freq = round(center_freqs[i], 1)
            gain = round(gains[i], 2)
            output_list.append((freq, gain))
        
        print(f"Current Configuration (Freq Hz, Gain):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True
    print_btn.disabled = True

# Link Buttons
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Click 'Print Gains' to see the current curve.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Layout
    controls_row_1 = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, mode_btn, print_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='5px')
    )
    
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row_1, vol_row, eq_box, out_log]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Click 'Print Gains' to see the current curve.
output underflow


In [3]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base "Safe" Volumes 
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 
slider_widgets_list = [] 

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    # --- Hard Mute Check ---
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=0.0, min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    slider_widgets_list.append(slider)
    
    # Freq Label
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='120px')
)

max_all_btn = widgets.Button(
    description='Max All',
    button_style='primary',
    icon='arrow-up',
    layout=widgets.Layout(width='100px')
)

min_all_btn = widgets.Button(
    description='Min All',
    button_style='primary',
    icon='arrow-down',
    layout=widgets.Layout(width='100px')
)

print_btn = widgets.Button(
    description='Print Gains',
    button_style='success',
    icon='print',
    layout=widgets.Layout(width='120px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

# Output area for printing
out_log = widgets.Output()

# --- Logic ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    for s in slider_widgets_list:
        s.value = 5.0

def set_min_all(b):
    for s in slider_widgets_list:
        s.value = 0.0

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = []
        for i in range(NUM_BANDS):
            # Format: (Frequency Hz, Gain)
            # Explicit conversion to python float to remove numpy tags
            freq = float(round(center_freqs[i], 1))
            gain = float(round(gains[i], 2))
            output_list.append((freq, gain))
        
        print(f"Current Configuration (Freq Hz, Gain):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True
    print_btn.disabled = True

# Link Buttons
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Click 'Print Gains' to see the current curve.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Layout
    controls_row_1 = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, mode_btn, print_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='5px')
    )
    
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row_1, vol_row, eq_box, out_log]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Click 'Print Gains' to see the current curve.
output underflow
output underflow


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base "Safe" Volumes 
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

# --- USER PRESET (INITIAL CONFIGURATION) ---
# Format: List of (Frequency, Gain) tuples
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 
slider_widgets_list = [] 

# --- Apply Initial Config to Global State ---
# We iterate through the config and set the gain array immediately
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        # Tuple is (freq, gain), we take the gain [1]
        gains[i] = INITIAL_CONFIG[i][1]
else:
    print(f"Warning: INITIAL_CONFIG length ({len(INITIAL_CONFIG)}) does not match NUM_BANDS ({NUM_BANDS}). Starting flat.")

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    # Determine Initial Value from Config or Array
    start_val = gains[i] # This is already set from INITIAL_CONFIG above

    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box
    gain_box = widgets.BoundedFloatText(
        value=start_val, 
        min=0.0, max=5.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=start_val, 
        min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    slider_widgets_list.append(slider)
    
    # Freq Label
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='120px')
)

max_all_btn = widgets.Button(
    description='Max All',
    button_style='primary',
    icon='arrow-up',
    layout=widgets.Layout(width='100px')
)

min_all_btn = widgets.Button(
    description='Min All',
    button_style='primary',
    icon='arrow-down',
    layout=widgets.Layout(width='100px')
)

print_btn = widgets.Button(
    description='Print Gains',
    button_style='success',
    icon='print',
    layout=widgets.Layout(width='120px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

out_log = widgets.Output()

# --- Logic ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    for s in slider_widgets_list:
        s.value = 5.0

def set_min_all(b):
    for s in slider_widgets_list:
        s.value = 0.0

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = []
        for i in range(NUM_BANDS):
            freq = float(round(center_freqs[i], 1))
            gain = float(round(gains[i], 2))
            output_list.append((freq, gain))
        
        print(f"Current Configuration (Freq Hz, Gain):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True
    print_btn.disabled = True

master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System with INITIAL_CONFIG loaded.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls_row_1 = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, mode_btn, print_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='5px')
    )
    
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row_1, vol_row, eq_box, out_log]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System with INITIAL_CONFIG loaded.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base Volumes (Scaled by slider value 0-10)
# Note: With sliders at 10, total output can be high. Use Master Vol to manage.
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05

# --- USER PRESET (INITIAL CONFIGURATION) ---
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
is_tonal_mode = False 
global_volume = 1.0 
slider_widgets_list = [] 

# --- Apply Initial Config ---
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        gains[i] = INITIAL_CONFIG[i][1]
else:
    print(f"Warning: INITIAL_CONFIG mismatch. Starting flat.")

# --- Frequency Calculations ---
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    global phases
    if status:
        print(status)
    
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    outdata.fill(0)
    
    # === MODE A: TONAL ONLY ===
    if is_tonal_mode:
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # === MODE B: NOISE ONLY ===
    else:
        current_noise_amp = BASE_NOISE_VOL * global_volume
        noise = np.random.uniform(-1, 1, size=frames) * current_noise_amp
        
        try:
            wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
                
            processed_signal = wp.reconstruct(update=False)
            
            if len(processed_signal) > frames:
                processed_signal = processed_signal[:frames]
            elif len(processed_signal) < frames:
                processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                
            outdata[:, 0] = processed_signal
            
        except Exception as e:
            pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    start_val = gains[i] 

    # Mute Btn
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    # Gain Box (Max = 10.0)
    gain_box = widgets.BoundedFloatText(
        value=start_val, 
        min=0.0, max=10.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    # Slider (Max = 10.0)
    slider = widgets.FloatSlider(
        value=start_val, 
        min=0.0, max=10.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    
    slider_widgets_list.append(slider)
    
    # Freq Label
    freq_val = end_freqs[i]
    if freq_val >= 1000:
        freq_text = f"{freq_val/1000:.1f}k"
    else:
        freq_text = f"{int(freq_val)}"

    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

mode_btn = widgets.ToggleButton(
    value=False,
    description='Mode: Noise',
    button_style='info',
    icon='music',
    layout=widgets.Layout(width='120px')
)

max_all_btn = widgets.Button(
    description='Max All',
    button_style='primary',
    icon='arrow-up',
    layout=widgets.Layout(width='100px')
)

min_all_btn = widgets.Button(
    description='Min All',
    button_style='primary',
    icon='arrow-down',
    layout=widgets.Layout(width='100px')
)

print_btn = widgets.Button(
    description='Print Gains',
    button_style='success',
    icon='print',
    layout=widgets.Layout(width='120px')
)

stop_btn = widgets.Button(
    description='Stop System',
    button_style='warning',
    icon='square',
    layout=widgets.Layout(width='120px')
)

master_vol_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Master Vol:',
    orientation='horizontal',
    readout=True,
    readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

out_log = widgets.Output()

# --- Logic ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    
    if are_all_muted:
        master_mute_btn.description = "Unmute All"
        master_mute_btn.button_style = 'success'
        master_mute_btn.icon = 'volume-up'
    else:
        master_mute_btn.description = "Mute All"
        master_mute_btn.button_style = 'danger'
        master_mute_btn.icon = 'volume-off'
        
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    for s in slider_widgets_list:
        s.value = 10.0 # Updated to 10.0

def set_min_all(b):
    for s in slider_widgets_list:
        s.value = 0.0

def toggle_mode(change):
    global is_tonal_mode
    is_tonal_mode = change['new']
    global phases
    phases.fill(0)
    
    if is_tonal_mode:
        mode_btn.description = "Mode: Tonal"
        mode_btn.button_style = 'warning'
    else:
        mode_btn.description = "Mode: Noise"
        mode_btn.button_style = 'info'

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = []
        for i in range(NUM_BANDS):
            freq = float(round(center_freqs[i], 1))
            gain = float(round(gains[i], 2))
            output_list.append((freq, gain))
        
        print(f"Current Configuration (Freq Hz, Gain):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    mode_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True
    print_btn.disabled = True

master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
mode_btn.observe(toggle_mode, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Range is now 0.0 to 10.0.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    controls_row_1 = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, mode_btn, print_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='5px')
    )
    
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([controls_row_1, vol_row, eq_box, out_log]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Range is now 0.0 to 10.0.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
import soundfile as sf  # Requires: pip install soundfile
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base Volumes
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05
# File base volume (often files are already normalized, so we keep this usually at 1.0 or slightly lower to leave headroom for EQ)
BASE_FILE_VOL = 0.8 

# --- USER PRESET ---
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
global_volume = 1.0 
slider_widgets_list = [] 

# Audio Source State
# Options: 'Noise', 'Tone', 'File'
current_source_mode = 'Noise' 

# File State
file_data = None
file_play_head = 0

# Apply Initial Config
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        gains[i] = INITIAL_CONFIG[i][1]

# Frequency Calculations
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time_info, status):
    global phases, file_play_head
    if status:
        print(status)
    
    # Hard Mute
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    outdata.fill(0)
    
    # ==========================
    # MODE: TONAL (Additive Synthesis)
    # ==========================
    if current_source_mode == 'Tone':
        t = np.arange(frames) / SAMPLE_RATE
        output_signal = np.zeros(frames)
        current_vol_scale = BASE_TONAL_VOL * global_volume
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * current_vol_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                output_signal += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        outdata[:, 0] = output_signal
        return

    # ==========================
    # MODE: NOISE OR FILE (Wavelet EQ)
    # ==========================
    else:
        # 1. Determine Input Signal
        input_signal = None
        
        if current_source_mode == 'Noise':
            current_noise_amp = BASE_NOISE_VOL * global_volume
            input_signal = np.random.uniform(-1, 1, size=frames) * current_noise_amp
            
        elif current_source_mode == 'File':
            if file_data is None:
                outdata.fill(0)
                return
            
            # Looping logic for file playback
            chunk_size = frames
            total_samples = len(file_data)
            
            # Check if we have enough data left
            remaining = total_samples - file_play_head
            
            if remaining >= chunk_size:
                input_signal = file_data[file_play_head : file_play_head + chunk_size]
                file_play_head += chunk_size
            else:
                # Wrap around
                part1 = file_data[file_play_head:]
                file_play_head = chunk_size - len(part1)
                part2 = file_data[:file_play_head]
                input_signal = np.concatenate((part1, part2))
            
            # Apply Base Volume for File
            input_signal = input_signal * BASE_FILE_VOL * global_volume

        # 2. Apply Wavelet EQ (Analysis -> Gain -> Synthesis)
        if input_signal is not None:
            try:
                # Analysis
                wp = pywt.WaveletPacket(data=input_signal, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
                nodes = wp.get_level(6, order='freq')
                
                # Apply Gains
                count = min(len(nodes), NUM_BANDS)
                for i in range(count):
                    if band_active[i]:
                        nodes[i].data = nodes[i].data * gains[i]
                    else:
                        nodes[i].data = nodes[i].data * 0.0
                
                # Synthesis
                processed_signal = wp.reconstruct(update=False)
                
                # Length Fix
                if len(processed_signal) > frames:
                    processed_signal = processed_signal[:frames]
                elif len(processed_signal) < frames:
                    processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
                    
                outdata[:, 0] = processed_signal
                
            except Exception as e:
                # Fallback if WPD fails
                outdata[:, 0] = input_signal

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    start_val = gains[i] 
    
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    gain_box = widgets.BoundedFloatText(
        value=start_val, min=0.0, max=10.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    slider = widgets.FloatSlider(
        value=start_val, min=0.0, max=10.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    slider_widgets_list.append(slider)
    
    freq_val = end_freqs[i]
    freq_text = f"{freq_val/1000:.1f}k" if freq_val >= 1000 else f"{int(freq_val)}"
    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

# Source Selector
source_dropdown = widgets.Dropdown(
    options=['Noise', 'Tone', 'File'],
    value='Noise',
    description='Source:',
    layout=widgets.Layout(width='200px')
)

# File Controls
file_path_text = widgets.Text(
    value='test.wav',
    placeholder='path/to/file.wav',
    description='Path:',
    layout=widgets.Layout(width='300px')
)

load_file_btn = widgets.Button(
    description='Load File',
    button_style='info',
    icon='upload',
    layout=widgets.Layout(width='120px')
)

file_status_lbl = widgets.Label(value="No file loaded")

# Standard Controls
master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

max_all_btn = widgets.Button(description='Max All', button_style='primary', icon='arrow-up', layout=widgets.Layout(width='100px'))
min_all_btn = widgets.Button(description='Min All', button_style='primary', icon='arrow-down', layout=widgets.Layout(width='100px'))
print_btn = widgets.Button(description='Print Gains', button_style='success', icon='print', layout=widgets.Layout(width='120px'))
stop_btn = widgets.Button(description='Stop System', button_style='warning', icon='square', layout=widgets.Layout(width='120px'))

master_vol_slider = widgets.FloatSlider(
    value=1.0, min=0.0, max=1.0, step=0.01,
    description='Master Vol:',
    orientation='horizontal', readout=True, readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

out_log = widgets.Output()

# --- Logic ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    if are_all_muted:
        master_mute_btn.description, master_mute_btn.button_style, master_mute_btn.icon = "Unmute All", 'success', 'volume-up'
    else:
        master_mute_btn.description, master_mute_btn.button_style, master_mute_btn.icon = "Mute All", 'danger', 'volume-off'
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    for s in slider_widgets_list: s.value = 10.0

def set_min_all(b):
    for s in slider_widgets_list: s.value = 0.0

def change_source(change):
    global current_source_mode, phases
    current_source_mode = change['new']
    phases.fill(0) # Reset phases on switch
    # visual feedback
    if current_source_mode == 'File':
        file_path_text.disabled = False
        load_file_btn.disabled = False
    else:
        # Keep file controls visible but maybe visually de-emphasized? 
        # Standard logic: they can load a file anytime.
        pass

def load_file_action(b):
    global file_data, file_play_head
    path = file_path_text.value
    file_status_lbl.value = "Loading..."
    try:
        # Load file
        data, fs = sf.read(path)
        
        # Convert Stereo to Mono
        if data.ndim > 1:
            data = np.mean(data, axis=1)
            
        # Check Samplerate
        if fs != SAMPLE_RATE:
            file_status_lbl.value = f"Loaded (Warning: fs={fs}!=44100). Playing..."
        else:
            file_status_lbl.value = f"Loaded successfully ({len(data)/fs:.1f}s)."
            
        file_data = data.astype(np.float32)
        file_play_head = 0
        
        # Auto-switch to File mode
        source_dropdown.value = 'File'
        
    except Exception as e:
        file_status_lbl.value = f"Error: {str(e)[:50]}"

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = []
        for i in range(NUM_BANDS):
            freq = float(round(center_freqs[i], 1))
            gain = float(round(gains[i], 2))
            output_list.append((freq, gain))
        print(f"Current Configuration (Freq Hz, Gain):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True
    print_btn.disabled = True

# Connections
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
load_file_btn.on_click(load_file_action)

source_dropdown.observe(change_source, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Layout
    # Row 1: Source Selection & File Load
    file_row = widgets.HBox(
        [source_dropdown, file_path_text, load_file_btn, file_status_lbl],
        layout=widgets.Layout(align_items='center', margin='0px 0px 10px 0px')
    )
    
    # Row 2: Main Buttons
    controls_row = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, print_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='5px')
    )
    
    # Row 3: Volume
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([file_row, controls_row, vol_row, eq_box, out_log]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
import soundfile as sf  # Requires: pip install soundfile
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base Volumes (Pre-Mixer)
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05
BASE_FILE_VOL = 0.8 

# --- USER PRESET ---
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
global_volume = 1.0 
slider_widgets_list = [] 

# Mix State (0.0 - 1.0)
mix_noise = 1.0
mix_tone = 0.0
mix_file = 0.0

# File State
file_data = None
file_play_head = 0

# Apply Initial Config
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        gains[i] = INITIAL_CONFIG[i][1]

# Frequency Calculations
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time_info, status):
    global phases, file_play_head
    if status:
        print(status)
    
    # Hard Mute Check
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    # Initialize buffers
    final_mix = np.zeros(frames)
    broadband_signal = np.zeros(frames) # Will hold Noise + File
    has_broadband_content = False

    # ---------------------------
    # A. Generate Tone Component
    # ---------------------------
    if mix_tone > 0.01:
        t = np.arange(frames) / SAMPLE_RATE
        tone_accum = np.zeros(frames)
        # Scale volume by mixer slider
        tone_scale = BASE_TONAL_VOL * mix_tone 
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                # Gain is applied here (Additive Synthesis)
                gain = gains[i] * tone_scale
                
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                sine_wave = gain * np.sin(2 * np.pi * freq * t + phases[i])
                tone_accum += sine_wave
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        
        final_mix += tone_accum

    # ---------------------------
    # B. Generate Broadband Component (Noise + File)
    # ---------------------------
    
    # B1. Noise
    if mix_noise > 0.01:
        noise_amp = BASE_NOISE_VOL * mix_noise
        broadband_signal += np.random.uniform(-1, 1, size=frames) * noise_amp
        has_broadband_content = True

    # B2. File
    if mix_file > 0.01 and file_data is not None:
        chunk_size = frames
        total_samples = len(file_data)
        remaining = total_samples - file_play_head
        
        file_chunk = None
        
        if remaining >= chunk_size:
            file_chunk = file_data[file_play_head : file_play_head + chunk_size]
            file_play_head += chunk_size
        else:
            part1 = file_data[file_play_head:]
            file_play_head = chunk_size - len(part1)
            part2 = file_data[:file_play_head]
            file_chunk = np.concatenate((part1, part2))
            
        if file_chunk is not None:
            broadband_signal += file_chunk * BASE_FILE_VOL * mix_file
            has_broadband_content = True

    # ---------------------------
    # C. Apply Wavelet EQ to Broadband
    # ---------------------------
    if has_broadband_content:
        try:
            wp = pywt.WaveletPacket(data=broadband_signal, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
            nodes = wp.get_level(6, order='freq')
            
            count = min(len(nodes), NUM_BANDS)
            for i in range(count):
                if band_active[i]:
                    nodes[i].data = nodes[i].data * gains[i]
                else:
                    nodes[i].data = nodes[i].data * 0.0
            
            processed_broadband = wp.reconstruct(update=False)
            
            # Length Fix
            if len(processed_broadband) > frames:
                processed_broadband = processed_broadband[:frames]
            elif len(processed_broadband) < frames:
                processed_broadband = np.pad(processed_broadband, (0, frames - len(processed_broadband)))
                
            final_mix += processed_broadband
            
        except Exception:
            # Fallback if WPD fails
            final_mix += broadband_signal

    # ---------------------------
    # D. Output
    # ---------------------------
    outdata[:, 0] = final_mix * global_volume


# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    if is_active:
        btn.icon = 'volume-up'
        btn.button_style = 'success'
    else:
        btn.icon = 'volume-off'
        btn.button_style = 'danger'

for i in range(NUM_BANDS):
    start_val = gains[i] 
    
    mute_btn = widgets.ToggleButton(
        value=True,
        button_style='success',
        icon='volume-up',
        layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px')
    )
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    gain_box = widgets.BoundedFloatText(
        value=start_val, min=0.0, max=10.0, step=0.01,
        layout=widgets.Layout(width='45px', height='30px')
    )
    
    slider = widgets.FloatSlider(
        value=start_val, min=0.0, max=10.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    slider_widgets_list.append(slider)
    
    freq_val = end_freqs[i]
    freq_text = f"{freq_val/1000:.1f}k" if freq_val >= 1000 else f"{int(freq_val)}"
    freq_lbl = widgets.Label(
        value=freq_text, 
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    col = widgets.VBox(
        [mute_btn, gain_box, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---

# File Controls
file_path_text = widgets.Text(
    value='test.wav',
    placeholder='path/to/file.wav',
    description='File Path:',
    layout=widgets.Layout(width='300px')
)

load_file_btn = widgets.Button(
    description='Load File',
    button_style='info',
    icon='upload',
    layout=widgets.Layout(width='120px')
)

file_status_lbl = widgets.Label(value="No file loaded")

# MIXER SLIDERS
mix_noise_slider = widgets.FloatSlider(
    value=1.0, min=0.0, max=1.0, step=0.01,
    description='Noise Mix:',
    orientation='horizontal',
    layout=widgets.Layout(width='300px')
)

mix_tone_slider = widgets.FloatSlider(
    value=0.0, min=0.0, max=1.0, step=0.01,
    description='Tone Mix:',
    orientation='horizontal',
    layout=widgets.Layout(width='300px')
)

mix_file_slider = widgets.FloatSlider(
    value=0.0, min=0.0, max=1.0, step=0.01,
    description='File Mix:',
    orientation='horizontal',
    layout=widgets.Layout(width='300px')
)

# Standard Controls
master_mute_btn = widgets.Button(
    description='Mute All',
    button_style='danger',
    icon='volume-off',
    layout=widgets.Layout(width='100px')
)

max_all_btn = widgets.Button(description='Max All', button_style='primary', icon='arrow-up', layout=widgets.Layout(width='100px'))
min_all_btn = widgets.Button(description='Min All', button_style='primary', icon='arrow-down', layout=widgets.Layout(width='100px'))
print_btn = widgets.Button(description='Print Gains', button_style='success', icon='print', layout=widgets.Layout(width='120px'))
stop_btn = widgets.Button(description='Stop System', button_style='warning', icon='square', layout=widgets.Layout(width='120px'))

master_vol_slider = widgets.FloatSlider(
    value=1.0, min=0.0, max=1.0, step=0.01,
    description='Master Vol:',
    orientation='horizontal', readout=True, readout_format='.0%', 
    layout=widgets.Layout(width='400px')
)

out_log = widgets.Output()

# --- Logic ---
are_all_muted = False

def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target_value = not are_all_muted
    if are_all_muted:
        master_mute_btn.description, master_mute_btn.button_style, master_mute_btn.icon = "Unmute All", 'success', 'volume-up'
    else:
        master_mute_btn.description, master_mute_btn.button_style, master_mute_btn.icon = "Mute All", 'danger', 'volume-off'
    for btn in band_mute_widgets:
        btn.value = target_value

def set_max_all(b):
    for s in slider_widgets_list: s.value = 10.0

def set_min_all(b):
    for s in slider_widgets_list: s.value = 0.0

# Mixer Updaters
def update_mix_noise(change):
    global mix_noise
    mix_noise = change['new']

def update_mix_tone(change):
    global mix_tone
    mix_tone = change['new']

def update_mix_file(change):
    global mix_file
    mix_file = change['new']

def load_file_action(b):
    global file_data, file_play_head
    path = file_path_text.value
    file_status_lbl.value = "Loading..."
    try:
        data, fs = sf.read(path)
        if data.ndim > 1: data = np.mean(data, axis=1) # Mono
        
        file_data = data.astype(np.float32)
        file_play_head = 0
        
        if fs != SAMPLE_RATE:
            file_status_lbl.value = f"Loaded (Resample Needed: {fs} vs {SAMPLE_RATE})."
        else:
            file_status_lbl.value = f"Loaded ({len(data)/fs:.1f}s)."
            
    except Exception as e:
        file_status_lbl.value = f"Error: {str(e)[:50]}"

def update_master_vol(change):
    global global_volume
    global_volume = change['new']

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = []
        for i in range(NUM_BANDS):
            freq = float(round(center_freqs[i], 1))
            gain = float(round(gains[i], 2))
            output_list.append((freq, gain))
        print(f"Current Configuration (Freq Hz, Gain):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("System Stopped.")
    stop_btn.disabled = True
    master_mute_btn.disabled = True
    master_vol_slider.disabled = True
    max_all_btn.disabled = True
    min_all_btn.disabled = True
    print_btn.disabled = True

# Connections
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
load_file_btn.on_click(load_file_action)

mix_noise_slider.observe(update_mix_noise, names='value')
mix_tone_slider.observe(update_mix_tone, names='value')
mix_file_slider.observe(update_mix_file, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- 4. Start Execution ---
print(f"Starting System. Use Mixer Sliders to blend sources.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    sd.stop()
    time.sleep(0.2) 

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    # Layout
    
    # Section 1: File Loading
    file_row = widgets.HBox([file_path_text, load_file_btn, file_status_lbl], layout=widgets.Layout(align_items='center', margin='0px 0px 15px 0px'))
    
    # Section 2: Mixer
    mixer_row = widgets.VBox([
        widgets.Label("<b>Source Mixer</b>"),
        mix_noise_slider,
        mix_tone_slider,
        mix_file_slider
    ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='0px 0px 15px 0px'))
    
    # Section 3: Global Controls
    controls_row = widgets.HBox(
        [master_mute_btn, max_all_btn, min_all_btn, print_btn, stop_btn],
        layout=widgets.Layout(justify_content='flex-start', gap='5px')
    )
    
    vol_row = widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px 20px 0px'))
    
    display(widgets.VBox([file_row, mixer_row, controls_row, vol_row, eq_box, out_log]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting System. Use Mixer Sliders to blend sources.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
import soundfile as sf
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base Volumes
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05
BASE_FILE_VOL = 0.8 

# --- USER PRESET ---
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
global_volume = 1.0 
slider_widgets_list = [] 

# Mix State (0.0 - 1.0)
mix_noise = 1.0
mix_tone = 0.0
mix_file = 0.0

# Processing Method: 'Wavelet' or 'FFT'
process_method = 'Wavelet'

# File State
file_data = None
file_play_head = 0

# Apply Initial Config
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        gains[i] = INITIAL_CONFIG[i][1]

# Frequency Calculations
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time_info, status):
    global phases, file_play_head
    if status:
        print(status)
    
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    final_mix = np.zeros(frames)
    broadband_signal = np.zeros(frames)
    has_broadband_content = False

    # A. Generate Tone (Additive - unaffected by FFT/Wavelet switch)
    if mix_tone > 0.01:
        t = np.arange(frames) / SAMPLE_RATE
        tone_accum = np.zeros(frames)
        tone_scale = BASE_TONAL_VOL * mix_tone 
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * tone_scale
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                tone_accum += gain * np.sin(2 * np.pi * freq * t + phases[i])
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        final_mix += tone_accum

    # B. Generate Broadband Source (Noise + File)
    if mix_noise > 0.01:
        noise_amp = BASE_NOISE_VOL * mix_noise
        broadband_signal += np.random.uniform(-1, 1, size=frames) * noise_amp
        has_broadband_content = True

    if mix_file > 0.01 and file_data is not None:
        chunk_size = frames
        total_samples = len(file_data)
        remaining = total_samples - file_play_head
        file_chunk = None
        
        if remaining >= chunk_size:
            file_chunk = file_data[file_play_head : file_play_head + chunk_size]
            file_play_head += chunk_size
        else:
            part1 = file_data[file_play_head:]
            file_play_head = chunk_size - len(part1)
            part2 = file_data[:file_play_head]
            file_chunk = np.concatenate((part1, part2))
            
        if file_chunk is not None:
            broadband_signal += file_chunk * BASE_FILE_VOL * mix_file
            has_broadband_content = True

    # C. Process Broadband (FFT vs Wavelet)
    if has_broadband_content:
        processed_broadband = np.zeros(frames)
        
        # --- OPTION 1: WAVELET PACKETS ---
        if process_method == 'Wavelet':
            try:
                wp = pywt.WaveletPacket(data=broadband_signal, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
                nodes = wp.get_level(6, order='freq')
                count = min(len(nodes), NUM_BANDS)
                for i in range(count):
                    g = gains[i] if band_active[i] else 0.0
                    nodes[i].data = nodes[i].data * g
                processed_broadband = wp.reconstruct(update=False)
            except:
                processed_broadband = broadband_signal
        
        # --- OPTION 2: FFT ---
        elif process_method == 'FFT':
            # Perform Real FFT
            spectrum = np.fft.rfft(broadband_signal)
            num_bins = len(spectrum)
            
            # Create a Gain Mask
            # We map the 64 Bands linearly to the FFT bins
            gain_mask = np.zeros(num_bins)
            bins_per_band = num_bins / NUM_BANDS
            
            for i in range(NUM_BANDS):
                start_idx = int(i * bins_per_band)
                end_idx = int((i + 1) * bins_per_band)
                # Ensure we don't go out of bounds
                if end_idx > num_bins: end_idx = num_bins
                
                g = gains[i] if band_active[i] else 0.0
                gain_mask[start_idx:end_idx] = g
                
            # Apply Filter
            spectrum *= gain_mask
            processed_broadband = np.fft.irfft(spectrum)

        # Fix Length (artifacts from padding/reconstruction)
        if len(processed_broadband) > frames:
            processed_broadband = processed_broadband[:frames]
        elif len(processed_broadband) < frames:
            processed_broadband = np.pad(processed_broadband, (0, frames - len(processed_broadband)))

        final_mix += processed_broadband

    # D. Output
    outdata[:, 0] = final_mix * global_volume


# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    btn.icon = 'volume-up' if is_active else 'volume-off'
    btn.button_style = 'success' if is_active else 'danger'

for i in range(NUM_BANDS):
    start_val = gains[i]
    mute_btn = widgets.ToggleButton(value=True, button_style='success', icon='volume-up', layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px'))
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    gain_box = widgets.BoundedFloatText(value=start_val, min=0.0, max=10.0, step=0.01, layout=widgets.Layout(width='45px', height='30px'))
    slider = widgets.FloatSlider(value=start_val, min=0.0, max=10.0, step=0.01, orientation='vertical', readout=False, layout=widgets.Layout(width='20px', height='400px'))
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    slider_widgets_list.append(slider)
    
    freq_val = end_freqs[i]
    freq_text = f"{freq_val/1000:.1f}k" if freq_val >= 1000 else f"{int(freq_val)}"
    freq_lbl = widgets.Label(value=freq_text, layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px'))
    
    col = widgets.VBox([mute_btn, gain_box, slider, freq_lbl], layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---
file_path_text = widgets.Text(value='test.wav', placeholder='path/to/file.wav', description='File Path:', layout=widgets.Layout(width='300px'))
load_file_btn = widgets.Button(description='Load File', button_style='info', icon='upload', layout=widgets.Layout(width='120px'))
file_status_lbl = widgets.Label(value="No file loaded")

# Method Selector
method_selector = widgets.Dropdown(
    options=['Wavelet', 'FFT'],
    value='Wavelet',
    description='Method:',
    layout=widgets.Layout(width='200px')
)

mix_noise_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Noise Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))
mix_tone_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='Tone Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))
mix_file_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='File Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))

master_mute_btn = widgets.Button(description='Mute All', button_style='danger', icon='volume-off', layout=widgets.Layout(width='100px'))
max_all_btn = widgets.Button(description='Max All', button_style='primary', icon='arrow-up', layout=widgets.Layout(width='100px'))
min_all_btn = widgets.Button(description='Min All', button_style='primary', icon='arrow-down', layout=widgets.Layout(width='100px'))
print_btn = widgets.Button(description='Print Gains', button_style='success', icon='print', layout=widgets.Layout(width='120px'))
stop_btn = widgets.Button(description='Stop System', button_style='warning', icon='square', layout=widgets.Layout(width='120px'))

master_vol_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Master Vol:', orientation='horizontal', readout=True, readout_format='.0%', layout=widgets.Layout(width='400px'))
out_log = widgets.Output()

# --- Logic ---
are_all_muted = False
def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target = not are_all_muted
    master_mute_btn.description = "Unmute All" if are_all_muted else "Mute All"
    master_mute_btn.button_style = 'success' if are_all_muted else 'danger'
    master_mute_btn.icon = 'volume-up' if are_all_muted else 'volume-off'
    for btn in band_mute_widgets: btn.value = target

def set_max_all(b):
    for s in slider_widgets_list: s.value = 10.0
def set_min_all(b):
    for s in slider_widgets_list: s.value = 0.0

def update_mix_noise(change): global mix_noise; mix_noise = change['new']
def update_mix_tone(change): global mix_tone; mix_tone = change['new']
def update_mix_file(change): global mix_file; mix_file = change['new']
def update_method(change): global process_method; process_method = change['new']
def update_master_vol(change): global global_volume; global_volume = change['new']

def load_file_action(b):
    global file_data, file_play_head
    try:
        data, fs = sf.read(file_path_text.value)
        if data.ndim > 1: data = np.mean(data, axis=1)
        file_data = data.astype(np.float32)
        file_play_head = 0
        file_status_lbl.value = f"Loaded ({len(data)/fs:.1f}s)."
    except Exception as e: file_status_lbl.value = f"Error: {str(e)[:50]}"

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = [(float(round(center_freqs[i], 1)), float(round(gains[i], 2))) for i in range(NUM_BANDS)]
        print(f"Current Configuration ({process_method} Mode):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream: global_stream.stop(); global_stream.close()
    print("System Stopped.")
    stop_btn.disabled = True

# Connections
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
load_file_btn.on_click(load_file_action)

mix_noise_slider.observe(update_mix_noise, names='value')
mix_tone_slider.observe(update_mix_tone, names='value')
mix_file_slider.observe(update_mix_file, names='value')
method_selector.observe(update_method, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- Start ---
print(f"Starting System.")
try:
    if global_stream: global_stream.stop(); global_stream.close()
    sd.stop(); time.sleep(0.2)
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE, blocksize=BLOCK_SIZE, channels=1, callback=audio_callback)
    global_stream.start()
    
    file_row = widgets.HBox([file_path_text, load_file_btn, file_status_lbl], layout=widgets.Layout(align_items='center', margin='0px 0px 15px 0px'))
    mixer_row = widgets.VBox([
        widgets.Label("<b>Source Mixer & Processing</b>"),
        method_selector,
        mix_noise_slider,
        mix_tone_slider,
        mix_file_slider
    ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='0px 0px 15px 0px'))
    
    controls = widgets.HBox([master_mute_btn, max_all_btn, min_all_btn, print_btn, stop_btn], layout=widgets.Layout(gap='5px'))
    display(widgets.VBox([file_row, mixer_row, controls, widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px')), eq_box, out_log]))
except Exception as e: print(f"Error: {e}")

Starting System.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
import soundfile as sf
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
OVERLAP_SIZE = 1024  # Size of the history buffer for Overlap-Save
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base Volumes
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05
BASE_FILE_VOL = 0.8 

# --- USER PRESET ---
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

global_stream = None 

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
global_volume = 1.0 
slider_widgets_list = [] 

# Mix State
mix_noise = 1.0
mix_tone = 0.0
mix_file = 0.0

# Processing
process_method = 'Wavelet'

# File State
file_data = None
file_play_head = 0

# --- NEW: Overlap Buffer ---
# We store the last chunk of raw input here to prepend to the next block
input_overlap_buffer = np.zeros(OVERLAP_SIZE, dtype=np.float32)

# Apply Initial Config
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        gains[i] = INITIAL_CONFIG[i][1]

# Frequency Calculations
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time_info, status):
    global phases, file_play_head, input_overlap_buffer
    if status:
        print(status)
    
    if global_volume <= 0.001:
        outdata.fill(0)
        return

    # Final output accumulator
    final_mix = np.zeros(frames)
    
    # === A. Tonal Component (Additive Synthesis) ===
    # Tonal synthesis is continuous (phase preserved) so it doesn't need Overlap-Save
    if mix_tone > 0.01:
        t = np.arange(frames) / SAMPLE_RATE
        tone_accum = np.zeros(frames)
        tone_scale = BASE_TONAL_VOL * mix_tone 
        
        for i in range(NUM_BANDS):
            if band_active[i] and gains[i] > 0.01:
                freq = center_freqs[i]
                gain = gains[i] * tone_scale
                phase_increment = 2 * np.pi * freq * (frames / SAMPLE_RATE)
                tone_accum += gain * np.sin(2 * np.pi * freq * t + phases[i])
                phases[i] = (phases[i] + phase_increment) % (2 * np.pi)
        final_mix += tone_accum

    # === B. Broadband Component (Noise + File) ===
    # This requires Overlap-Save to avoid filter glitches
    
    # 1. Generate Current Chunk Raw Data
    raw_current = np.zeros(frames)
    has_content = False

    if mix_noise > 0.01:
        noise_amp = BASE_NOISE_VOL * mix_noise
        raw_current += np.random.uniform(-1, 1, size=frames) * noise_amp
        has_content = True

    if mix_file > 0.01 and file_data is not None:
        chunk_size = frames
        total_samples = len(file_data)
        remaining = total_samples - file_play_head
        file_chunk = None
        
        if remaining >= chunk_size:
            file_chunk = file_data[file_play_head : file_play_head + chunk_size]
            file_play_head += chunk_size
        else:
            part1 = file_data[file_play_head:]
            file_play_head = chunk_size - len(part1)
            part2 = file_data[:file_play_head]
            file_chunk = np.concatenate((part1, part2))
            
        if file_chunk is not None:
            raw_current += file_chunk * BASE_FILE_VOL * mix_file
            has_content = True

    # 2. Apply Overlap-Save Processing
    if has_content:
        # Construct Extended Signal: [History (Overlap)] + [Current]
        extended_signal = np.concatenate((input_overlap_buffer, raw_current))
        
        # Save the end of current chunk for the NEXT iteration
        input_overlap_buffer = raw_current[-OVERLAP_SIZE:]
        
        processed_extended = np.zeros_like(extended_signal)
        
        # --- Method 1: Wavelet Packets ---
        if process_method == 'Wavelet':
            try:
                # We process the FULL extended signal
                wp = pywt.WaveletPacket(data=extended_signal, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
                nodes = wp.get_level(6, order='freq')
                
                count = min(len(nodes), NUM_BANDS)
                for i in range(count):
                    g = gains[i] if band_active[i] else 0.0
                    nodes[i].data = nodes[i].data * g
                    
                processed_extended = wp.reconstruct(update=False)
            except:
                processed_extended = extended_signal

        # --- Method 2: FFT ---
        elif process_method == 'FFT':
            # FFT of extended signal (Linear Convolution setup)
            spectrum = np.fft.rfft(extended_signal)
            num_bins = len(spectrum)
            
            # Create Gain Mask
            gain_mask = np.zeros(num_bins)
            bins_per_band = num_bins / NUM_BANDS
            
            for i in range(NUM_BANDS):
                start_idx = int(i * bins_per_band)
                end_idx = int((i + 1) * bins_per_band)
                if end_idx > num_bins: end_idx = num_bins
                g = gains[i] if band_active[i] else 0.0
                gain_mask[start_idx:end_idx] = g
                
            spectrum *= gain_mask
            processed_extended = np.fft.irfft(spectrum)

        # 3. Slice and Extract Valid Output
        # Due to padding/reconstruction, ensure length match
        if len(processed_extended) != len(extended_signal):
             # Basic resize if reconstruction changed length slightly
             processed_extended = np.resize(processed_extended, len(extended_signal))
        
        # The first OVERLAP_SIZE samples are "contaminated" by the previous block's tail/transients
        # We discard them and keep the part corresponding to 'raw_current'
        valid_part = processed_extended[OVERLAP_SIZE:]
        
        # Safety check for length match with output frame
        if len(valid_part) > frames:
            valid_part = valid_part[:frames]
        elif len(valid_part) < frames:
            valid_part = np.pad(valid_part, (0, frames - len(valid_part)))
            
        final_mix += valid_part

    # Output
    outdata[:, 0] = final_mix * global_volume


# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    btn.icon = 'volume-up' if is_active else 'volume-off'
    btn.button_style = 'success' if is_active else 'danger'

for i in range(NUM_BANDS):
    start_val = gains[i]
    mute_btn = widgets.ToggleButton(value=True, button_style='success', icon='volume-up', layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px'))
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    gain_box = widgets.BoundedFloatText(value=start_val, min=0.0, max=10.0, step=0.01, layout=widgets.Layout(width='45px', height='30px'))
    slider = widgets.FloatSlider(value=start_val, min=0.0, max=10.0, step=0.01, orientation='vertical', readout=False, layout=widgets.Layout(width='20px', height='400px'))
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    slider_widgets_list.append(slider)
    
    freq_val = end_freqs[i]
    freq_text = f"{freq_val/1000:.1f}k" if freq_val >= 1000 else f"{int(freq_val)}"
    freq_lbl = widgets.Label(value=freq_text, layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px'))
    
    col = widgets.VBox([mute_btn, gain_box, slider, freq_lbl], layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---
file_path_text = widgets.Text(value='test.wav', placeholder='path/to/file.wav', description='File Path:', layout=widgets.Layout(width='300px'))
load_file_btn = widgets.Button(description='Load File', button_style='info', icon='upload', layout=widgets.Layout(width='120px'))
file_status_lbl = widgets.Label(value="No file loaded")

# Method Selector
method_selector = widgets.Dropdown(
    options=['Wavelet', 'FFT'],
    value='Wavelet',
    description='Method:',
    layout=widgets.Layout(width='200px')
)

mix_noise_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Noise Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))
mix_tone_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='Tone Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))
mix_file_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='File Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))

master_mute_btn = widgets.Button(description='Mute All', button_style='danger', icon='volume-off', layout=widgets.Layout(width='100px'))
max_all_btn = widgets.Button(description='Max All', button_style='primary', icon='arrow-up', layout=widgets.Layout(width='100px'))
min_all_btn = widgets.Button(description='Min All', button_style='primary', icon='arrow-down', layout=widgets.Layout(width='100px'))
print_btn = widgets.Button(description='Print Gains', button_style='success', icon='print', layout=widgets.Layout(width='120px'))
stop_btn = widgets.Button(description='Stop System', button_style='warning', icon='square', layout=widgets.Layout(width='120px'))

master_vol_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Master Vol:', orientation='horizontal', readout=True, readout_format='.0%', layout=widgets.Layout(width='400px'))
out_log = widgets.Output()

# --- Logic ---
are_all_muted = False
def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target = not are_all_muted
    master_mute_btn.description = "Unmute All" if are_all_muted else "Mute All"
    master_mute_btn.button_style = 'success' if are_all_muted else 'danger'
    master_mute_btn.icon = 'volume-up' if are_all_muted else 'volume-off'
    for btn in band_mute_widgets: btn.value = target

def set_max_all(b):
    for s in slider_widgets_list: s.value = 10.0
def set_min_all(b):
    for s in slider_widgets_list: s.value = 0.0

def update_mix_noise(change): global mix_noise; mix_noise = change['new']
def update_mix_tone(change): global mix_tone; mix_tone = change['new']
def update_mix_file(change): global mix_file; mix_file = change['new']
def update_method(change): global process_method; process_method = change['new']
def update_master_vol(change): global global_volume; global_volume = change['new']

def load_file_action(b):
    global file_data, file_play_head
    try:
        data, fs = sf.read(file_path_text.value)
        if data.ndim > 1: data = np.mean(data, axis=1)
        file_data = data.astype(np.float32)
        file_play_head = 0
        file_status_lbl.value = f"Loaded ({len(data)/fs:.1f}s)."
    except Exception as e: file_status_lbl.value = f"Error: {str(e)[:50]}"

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = [(float(round(center_freqs[i], 1)), float(round(gains[i], 2))) for i in range(NUM_BANDS)]
        print(f"Current Configuration ({process_method} Mode):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream: global_stream.stop(); global_stream.close()
    print("System Stopped.")
    stop_btn.disabled = True

# Connections
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
load_file_btn.on_click(load_file_action)

mix_noise_slider.observe(update_mix_noise, names='value')
mix_tone_slider.observe(update_mix_tone, names='value')
mix_file_slider.observe(update_mix_file, names='value')
method_selector.observe(update_method, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- Start ---
print(f"Starting System with Overlap-Save (Anti-Glitch).")
try:
    if global_stream: global_stream.stop(); global_stream.close()
    sd.stop(); time.sleep(0.2)
    # Reset overlap buffer on start
    input_overlap_buffer = np.zeros(OVERLAP_SIZE, dtype=np.float32)
    
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE, blocksize=BLOCK_SIZE, channels=1, callback=audio_callback)
    global_stream.start()
    
    file_row = widgets.HBox([file_path_text, load_file_btn, file_status_lbl], layout=widgets.Layout(align_items='center', margin='0px 0px 15px 0px'))
    mixer_row = widgets.VBox([
        widgets.Label("<b>Source Mixer & Processing</b>"),
        method_selector,
        mix_noise_slider,
        mix_tone_slider,
        mix_file_slider
    ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='0px 0px 15px 0px'))
    
    controls = widgets.HBox([master_mute_btn, max_all_btn, min_all_btn, print_btn, stop_btn], layout=widgets.Layout(gap='5px'))
    display(widgets.VBox([file_row, mixer_row, controls, widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px')), eq_box, out_log]))
except Exception as e: print(f"Error: {e}")

Starting System with Overlap-Save (Anti-Glitch).


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
import soundfile as sf
from IPython.display import display, clear_output
import time

# --- Configuration ---
SAMPLE_RATE = 44100
# We decouple the processing block size from the audio callback size for stability
WINDOW_SIZE = 4096       # The size of the FFT/Wavelet window
HOP_SIZE = WINDOW_SIZE // 2  # 50% Overlap (Standard for OLA)
NUM_BANDS = 64
WAVELET_NAME = 'db1'

# Base Volumes
BASE_NOISE_VOL = 0.2
BASE_TONAL_VOL = 0.05
BASE_FILE_VOL = 0.8 

# --- USER PRESET ---
INITIAL_CONFIG = [
    (172.3, 0.0), (516.8, 0.0), (861.3, 0.0), (1205.9, 0.0), (1550.4, 0.0), 
    (1894.9, 0.0), (2239.5, 0.0), (2584.0, 0.0), (2928.5, 0.0), (3273.0, 0.0), 
    (3617.6, 0.0), (3962.1, 0.0), (4306.6, 0.0), (4651.2, 2.56), (4995.7, 0.0), 
    (5340.2, 0.0), (5684.8, 0.0), (6029.3, 0.0), (6373.8, 0.0), (6718.4, 0.0), 
    (7062.9, 0.0), (7407.4, 0.0), (7752.0, 0.0), (8096.5, 0.0), (8441.0, 0.0), 
    (8785.5, 0.0), (9130.1, 0.0), (9474.6, 0.0), (9819.1, 0.0), (10163.7, 0.0), 
    (10508.2, 0.0), (10852.7, 0.0), (11197.3, 0.0), (11541.8, 0.0), (11886.3, 0.0), 
    (12230.9, 0.0), (12575.4, 0.0), (12919.9, 0.0), (13264.5, 0.0), (13609.0, 0.0), 
    (13953.5, 0.0), (14298.0, 0.0), (14642.6, 0.0), (14987.1, 0.0), (15331.6, 0.0), 
    (15676.2, 0.0), (16020.7, 0.0), (16365.2, 0.0), (16709.8, 0.0), (17054.3, 0.0), 
    (17398.8, 0.0), (17743.4, 0.0), (18087.9, 0.0), (18432.4, 0.0), (18777.0, 0.0), 
    (19121.5, 0.0), (19466.0, 0.0), (19810.5, 0.0), (20155.1, 0.0), (20499.6, 0.0), 
    (20844.1, 0.0), (21188.7, 0.0), (21533.2, 0.0), (21877.7, 0.0)
]

# --- Global State ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)
band_active = np.ones(NUM_BANDS, dtype=bool) 
global_volume = 1.0 
slider_widgets_list = [] 
global_stream = None

# Mix State
mix_noise = 1.0
mix_tone = 0.0
mix_file = 0.0

# Processing
process_method = 'Wavelet'

# File State
file_data = None
file_play_head = 0

# Apply Initial Config
if len(INITIAL_CONFIG) == NUM_BANDS:
    for i in range(NUM_BANDS):
        gains[i] = INITIAL_CONFIG[i][1]

# Frequency Calculations
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS
end_freqs = np.array([(i + 1) * bandwidth for i in range(NUM_BANDS)])
center_freqs = np.array([(i * bandwidth) + (bandwidth / 2) for i in range(NUM_BANDS)])
phases = np.zeros(NUM_BANDS) 

# --- HELPER: Circular Buffer Logic ---
class AudioBuffer:
    def __init__(self, size):
        self.buffer = np.zeros(size, dtype=np.float32)
        self.write_idx = 0
        self.read_idx = 0
        self.count = 0
        self.size = size
    
    def write(self, data):
        n = len(data)
        if n > self.size: return # Safety
        
        # Wrap logic
        end_idx = (self.write_idx + n) % self.size
        if end_idx > self.write_idx:
            self.buffer[self.write_idx:end_idx] = data
        else:
            part1 = self.size - self.write_idx
            self.buffer[self.write_idx:] = data[:part1]
            self.buffer[:end_idx] = data[part1:]
            
        self.write_idx = end_idx
        self.count = min(self.count + n, self.size)

    def read_overlap_slice(self, length, offset_from_end):
        """Reads a slice of 'length' looking back from the write head"""
        start = (self.write_idx - offset_from_end) % self.size
        end = (start + length) % self.size
        
        if end > start:
            return self.buffer[start:end]
        else:
            return np.concatenate((self.buffer[start:], self.buffer[:end]))
            
    def pop(self, n):
        """Advances read head (conceptual, just reducing count for this logic)"""
        self.count -= n

# Buffers for OLA
# We need enough space to store incoming data and accumulating output
input_fifo = AudioBuffer(WINDOW_SIZE * 4)
output_fifo = AudioBuffer(WINDOW_SIZE * 4)

# The Hanning Window (The Key to Glitch-Free Audio)
hanning_window = np.hanning(WINDOW_SIZE)


# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time_info, status):
    global phases, file_play_head, input_fifo, output_fifo
    if status: print(status)
    
    outdata.fill(0)
    if global_volume <= 0.001: return

    # --- A. Prepare Input Audio (Noise + File) ---
    raw_input = np.zeros(frames, dtype=np.float32)
    
    # Noise
    if mix_noise > 0.01:
        raw_input += np.random.uniform(-1, 1, size=frames) * BASE_NOISE_VOL * mix_noise

    # File
    if mix_file > 0.01 and file_data is not None:
        chunk = None
        rem = len(file_data) - file_play_head
        if rem >= frames:
            chunk = file_data[file_play_head : file_play_head + frames]
            file_play_head += frames
        else:
            p1 = file_data[file_play_head:]
            file_play_head = frames - len(p1)
            p2 = file_data[:file_play_head]
            chunk = np.concatenate((p1, p2))
        
        if chunk is not None:
            raw_input += chunk * BASE_FILE_VOL * mix_file

    # --- B. Push to Input FIFO ---
    input_fifo.write(raw_input)
    
    # --- C. Process in Overlapping Blocks ---
    # While we have enough data in input_fifo to process a full window
    while input_fifo.count >= WINDOW_SIZE:
        # 1. Read one WINDOW_SIZE (Current + History)
        # We read from the "end" of the written data effectively
        # But standard OLA consumes HOP_SIZE new samples per step
        
        # We grab the oldest WINDOW_SIZE samples available in the buffer
        # Since 'count' tracks unconsumed samples:
        
        # Note: Implementing a ring buffer pointer logic is complex in snippets.
        # Simplification: We look at the accumulated count. 
        # We perform processing whenever we have accumulated HOP_SIZE new samples 
        # relative to the previous step.
        
        # ACTUALLY: Easier Pythonic way for the Loop:
        # Just grab the block relative to write_idx
        processing_block = input_fifo.read_overlap_slice(WINDOW_SIZE, input_fifo.count)
        
        # 2. Apply Window
        windowed_block = processing_block * hanning_window
        
        # 3. Process (FFT or Wavelet)
        processed_block = np.zeros_like(windowed_block)
        
        if process_method == 'FFT':
            spec = np.fft.rfft(windowed_block)
            n_bins = len(spec)
            mask = np.zeros(n_bins)
            bins_per_band = n_bins / NUM_BANDS
            for i in range(NUM_BANDS):
                if band_active[i]:
                    s = int(i * bins_per_band)
                    e = int((i + 1) * bins_per_band)
                    if e > n_bins: e = n_bins
                    mask[s:e] = gains[i]
            processed_block = np.fft.irfft(spec * mask, n=WINDOW_SIZE)
            
        elif process_method == 'Wavelet':
            try:
                wp = pywt.WaveletPacket(data=windowed_block, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
                nodes = wp.get_level(6, order='freq')
                c = min(len(nodes), NUM_BANDS)
                for i in range(c):
                    g = gains[i] if band_active[i] else 0.0
                    nodes[i].data *= g
                processed_block = wp.reconstruct(update=False)
                # Resize if reconstruction is slightly off
                if len(processed_block) != WINDOW_SIZE:
                    processed_block = np.resize(processed_block, WINDOW_SIZE)
            except:
                processed_block = windowed_block

        # 4. Overlap-Add to Output FIFO
        # We add this processed block into the output buffer at the correct position
        # output_fifo write pointer acts as the "current time"
        # We must add to the region: [write_idx - count : write_idx - count + WINDOW_SIZE]
        
        # Due to circular buffer complexity, here is the robust linear way using the buffer class:
        # We manually add to the buffer's array.
        
        # Where does this block belong?
        # It represents the audio starting at (Total_Time - Count).
        
        # Let's simplify: 
        # Output Buffer needs to be treated as a continuous rolling accumulator.
        # A simpler trick for this callback environment:
        # We read HOP_SIZE from input, we add WINDOW_SIZE to output accumulator.
        
        # Let's just consume HOP_SIZE from input_fifo
        input_fifo.count -= HOP_SIZE
        
        # Add to Output Accumulator (Manual Ring Buffer Math)
        start_pos = output_fifo.write_idx
        end_pos = (start_pos + WINDOW_SIZE) % output_fifo.size
        
        if end_pos > start_pos:
            output_fifo.buffer[start_pos:end_pos] += processed_block
        else:
            p1 = output_fifo.size - start_pos
            output_fifo.buffer[start_pos:] += processed_block[:p1]
            output_fifo.buffer[:end_pos] += processed_block[p1:]
            
        # Advance Output Write Pointer by HOP_SIZE
        output_fifo.write_idx = (output_fifo.write_idx + HOP_SIZE) % output_fifo.size
        output_fifo.count += HOP_SIZE # We have valid data available

    # --- D. Read from Output FIFO to DAC ---
    # We need 'frames' samples.
    if output_fifo.count >= frames:
        # Read from read_idx
        out_block = np.zeros(frames)
        start = output_fifo.read_idx
        end = (start + frames) % output_fifo.size
        
        if end > start:
            out_block = output_fifo.buffer[start:end]
            output_fifo.buffer[start:end] = 0 # Clear after read (Crucial for OLA!)
        else:
            p1 = output_fifo.size - start
            out_block = np.concatenate((output_fifo.buffer[start:], output_fifo.buffer[:end]))
            # Clear
            output_fifo.buffer[start:] = 0
            output_fifo.buffer[:end] = 0
            
        output_fifo.read_idx = end
        output_fifo.count -= frames
        
        # Add Tonal Component (Parallel, no OLA needed as it's pure sine)
        if mix_tone > 0.01:
            t = np.arange(frames) / SAMPLE_RATE
            tone_accum = np.zeros(frames)
            scale = BASE_TONAL_VOL * mix_tone
            for i in range(NUM_BANDS):
                if band_active[i] and gains[i] > 0.01:
                    freq = center_freqs[i]
                    tone_accum += (gains[i] * scale) * np.sin(2 * np.pi * freq * t + phases[i])
                    phases[i] = (phases[i] + 2*np.pi*freq*(frames/SAMPLE_RATE)) % (2*np.pi)
            out_block += tone_accum

        outdata[:, 0] = out_block * global_volume
    else:
        # Buffering... (silence to avoid garbage)
        pass

# --- 2. The UI Logic ---
slider_containers = []
band_mute_widgets = []

def update_gain(change):
    gains[change['owner'].band_index] = change['new']

def toggle_band_mute(change):
    btn = change['owner']
    idx = btn.band_index
    is_active = change['new']
    band_active[idx] = is_active
    btn.icon = 'volume-up' if is_active else 'volume-off'
    btn.button_style = 'success' if is_active else 'danger'

for i in range(NUM_BANDS):
    start_val = gains[i]
    mute_btn = widgets.ToggleButton(value=True, button_style='success', icon='volume-up', layout=widgets.Layout(width='30px', height='30px', margin='0px 0px 5px 0px'))
    mute_btn.band_index = i
    mute_btn.observe(toggle_band_mute, names='value')
    band_mute_widgets.append(mute_btn)

    gain_box = widgets.BoundedFloatText(value=start_val, min=0.0, max=10.0, step=0.01, layout=widgets.Layout(width='45px', height='30px'))
    slider = widgets.FloatSlider(value=start_val, min=0.0, max=10.0, step=0.01, orientation='vertical', readout=False, layout=widgets.Layout(width='20px', height='400px'))
    slider.band_index = i
    widgets.jslink((slider, 'value'), (gain_box, 'value'))
    slider.observe(update_gain, names='value')
    slider_widgets_list.append(slider)
    
    freq_val = end_freqs[i]
    freq_text = f"{freq_val/1000:.1f}k" if freq_val >= 1000 else f"{int(freq_val)}"
    freq_lbl = widgets.Label(value=freq_text, layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px'))
    
    col = widgets.VBox([mute_btn, gain_box, slider, freq_lbl], layout=widgets.Layout(align_items='center', margin='0px 4px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Master Controls ---
file_path_text = widgets.Text(value='test.wav', placeholder='path/to/file.wav', description='File Path:', layout=widgets.Layout(width='300px'))
load_file_btn = widgets.Button(description='Load File', button_style='info', icon='upload', layout=widgets.Layout(width='120px'))
file_status_lbl = widgets.Label(value="No file loaded")

# Method Selector
method_selector = widgets.Dropdown(
    options=['Wavelet', 'FFT'],
    value='Wavelet',
    description='Method:',
    layout=widgets.Layout(width='200px')
)

mix_noise_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Noise Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))
mix_tone_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='Tone Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))
mix_file_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='File Mix:', orientation='horizontal', layout=widgets.Layout(width='300px'))

master_mute_btn = widgets.Button(description='Mute All', button_style='danger', icon='volume-off', layout=widgets.Layout(width='100px'))
max_all_btn = widgets.Button(description='Max All', button_style='primary', icon='arrow-up', layout=widgets.Layout(width='100px'))
min_all_btn = widgets.Button(description='Min All', button_style='primary', icon='arrow-down', layout=widgets.Layout(width='100px'))
print_btn = widgets.Button(description='Print Gains', button_style='success', icon='print', layout=widgets.Layout(width='120px'))
stop_btn = widgets.Button(description='Stop System', button_style='warning', icon='square', layout=widgets.Layout(width='120px'))

master_vol_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Master Vol:', orientation='horizontal', readout=True, readout_format='.0%', layout=widgets.Layout(width='400px'))
out_log = widgets.Output()

# --- Logic ---
are_all_muted = False
def master_mute_action(b):
    global are_all_muted
    are_all_muted = not are_all_muted
    target = not are_all_muted
    master_mute_btn.description = "Unmute All" if are_all_muted else "Mute All"
    master_mute_btn.button_style = 'success' if are_all_muted else 'danger'
    master_mute_btn.icon = 'volume-up' if are_all_muted else 'volume-off'
    for btn in band_mute_widgets: btn.value = target

def set_max_all(b):
    for s in slider_widgets_list: s.value = 10.0
def set_min_all(b):
    for s in slider_widgets_list: s.value = 0.0

def update_mix_noise(change): global mix_noise; mix_noise = change['new']
def update_mix_tone(change): global mix_tone; mix_tone = change['new']
def update_mix_file(change): global mix_file; mix_file = change['new']
def update_method(change): global process_method; process_method = change['new']
def update_master_vol(change): global global_volume; global_volume = change['new']

def load_file_action(b):
    global file_data, file_play_head
    try:
        data, fs = sf.read(file_path_text.value)
        if data.ndim > 1: data = np.mean(data, axis=1)
        file_data = data.astype(np.float32)
        file_play_head = 0
        file_status_lbl.value = f"Loaded ({len(data)/fs:.1f}s)."
    except Exception as e: file_status_lbl.value = f"Error: {str(e)[:50]}"

def print_gains_action(b):
    with out_log:
        clear_output()
        output_list = [(float(round(center_freqs[i], 1)), float(round(gains[i], 2))) for i in range(NUM_BANDS)]
        print(f"Current Configuration ({process_method} Mode):")
        print(output_list)

def stop_stream(b):
    global global_stream
    if global_stream: global_stream.stop(); global_stream.close()
    print("System Stopped.")
    stop_btn.disabled = True

# Connections
master_mute_btn.on_click(master_mute_action)
max_all_btn.on_click(set_max_all)
min_all_btn.on_click(set_min_all)
stop_btn.on_click(stop_stream)
print_btn.on_click(print_gains_action)
load_file_btn.on_click(load_file_action)

mix_noise_slider.observe(update_mix_noise, names='value')
mix_tone_slider.observe(update_mix_tone, names='value')
mix_file_slider.observe(update_mix_file, names='value')
method_selector.observe(update_method, names='value')
master_vol_slider.observe(update_master_vol, names='value')

# --- Start ---
print(f"Starting System with TRUE Overlap-Add (Glitch-Free).")
try:
    if global_stream: global_stream.stop(); global_stream.close()
    sd.stop(); time.sleep(0.2)
    
    # Reset Buffers
    input_fifo = AudioBuffer(WINDOW_SIZE * 4)
    output_fifo = AudioBuffer(WINDOW_SIZE * 4)
    
    # Pre-fill output buffer slightly to prevent initial underrun
    output_fifo.write_idx = WINDOW_SIZE
    output_fifo.count = WINDOW_SIZE
    
    # Fix blocksize for the stream to ensure consistent processing latency
    # However, Python streams often prefer variable. We handle variable 'frames' in callback.
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE, channels=1, callback=audio_callback)
    global_stream.start()
    
    file_row = widgets.HBox([file_path_text, load_file_btn, file_status_lbl], layout=widgets.Layout(align_items='center', margin='0px 0px 15px 0px'))
    mixer_row = widgets.VBox([
        widgets.Label("<b>Source Mixer & Processing</b>"),
        method_selector,
        mix_noise_slider,
        mix_tone_slider,
        mix_file_slider
    ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='0px 0px 15px 0px'))
    
    controls = widgets.HBox([master_mute_btn, max_all_btn, min_all_btn, print_btn, stop_btn], layout=widgets.Layout(gap='5px'))
    display(widgets.VBox([file_row, mixer_row, controls, widgets.HBox([master_vol_slider], layout=widgets.Layout(margin='10px 0px')), eq_box, out_log]))
except Exception as e: print(f"Error: {e}")

Starting System with TRUE Overlap-Add (Glitch-Free).
